In [ ]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import numpy as np
import plotly.express as px
import dash_bootstrap_components as dbc
import regex as re
import geopandas as gpd

from datetime import datetime

##### Table of Contents
- <a href="#Loading-the-Dengue-Dataset">Loading the Dengue Dataset</a>
- <a href="#Cleaning-the-datasets">Cleaning the datasets</a>
- <a href="#Exploratory-Data-Analysis">Exploratory Data Analysis (EDA)</a>

---
### Loading the Dengue Dataset and Regional boundaries
<a href="#Table-of-Contents">Back to Table of Contents</a>

In [ ]:
path = "../datasets/csv/Dengue_Data_2020.csv"
df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
path = "../datasets/json/regions.json"
regions = gpd.read_file(path)

In [ ]:
regions

---
### Cleaning the datasets
<a href="#Table-of-Contents">Back to Table of Contents</a>

#### Compiling provincial boundary data

In [ ]:
r_codes = [
    "100000000", "200000000", "300000000", "400000000", "500000000", "600000000", "700000000",
    "800000000", "900000000", "1000000000", "1100000000", "1200000000", "1300000000", "1400000000",
    "1600000000", "1700000000", "1900000000"
]

In [ ]:
# Preview of one file (this file in paricular shows provinces of the Ilocos Region)
path = "../datasets/json/provinces_part/provdists-region-" + r_codes[0] + ".0.1.json"
gdf = gpd.read_file(path)
gdf.head()

In [ ]:
r_codes = r_codes[1:]

for i in range(len(r_codes)):
    to_add = gpd.read_file("../datasets/json/provinces_part/provdists-region-" + r_codes[i] + ".0.1.json")
    gdf = pd.concat([gdf, to_add], ignore_index=True)

In [ ]:
gdf

In [ ]:
# Renaming the last item, which is the Special Geographic Area found neighboring Cotabato
gdf.at[87, "adm2_en"] = "Special Geographic Area"
gdf.at[87, "geo_level"] = "Prov"

In [ ]:
gdf.to_file("../datasets/json/provinces.json", driver="GeoJSON")

In [ ]:
# Checking
path = "../datasets/json/provinces.json"
provinces = gpd.read_file(path)
provinces.shape

#### Dummy header

In [ ]:
df = df.drop([0]).reset_index().drop("index", axis=1)
df.head()

#### Numerical values are saved as string

In [ ]:
df.dtypes

In [ ]:
df["cases"] = pd.to_numeric(df["cases"])
df["deaths"] = pd.to_numeric(df["deaths"])
df["year"] = pd.to_numeric(df["year"])
df.dtypes

#### Date values have inconsistent format

In [ ]:
df["date"].unique()

In [ ]:
day_pattern = '(?<=\d\/0)\d(?=\/)'
month_pattern = '\d+(?=\/\d+\/)'
year_pattern = '(?<=\d+\/\d+\/)\d+'

dates = df["date"].to_list()

for i in range(len(dates)):
    match = re.search(day_pattern, dates[i])
    if match is not None:
        day = match.group()
        month = re.search(month_pattern, dates[i]).group()
        year = re.search(year_pattern, dates[i]).group()
        
        new_date = month + "/" + day + "/" + year
        dates[i] = new_date

df["date"] = dates
df["date"].unique()

#### Inconsistent format for region values

In [ ]:
df.rename(columns = {'Region':'region'}, inplace = True)
df.head()

In [ ]:
# Unique values in 'region' column in Dengue dataset
df["region"].unique()

In [ ]:
# Proper region names, as per our GeoJSONs
regions["adm1_en"].unique()

In [ ]:
# Renaming each value to the corresponding format found in our GeoJSONs, for consistency
df["region"] = df["region"].map({
    'Region I-ILOCOS REGION': "Region I (Ilocos Region)",
    'Region II-CAGAYAN VALLEY': "Region II (Cagayan Valley)",
    'REGION III-CENTRAL LUZON': "Region III (Central Luzon)",
    'REGION IV-A-CALABARZON': "Region IV-A (CALABARZON)",
    'REGION IVB-MIMAROPA': "MIMAROPA Region",
    'REGION V-BICOL REGION': "Region V (Bicol Region)",
    'REGION VI-WESTERN VISAYAS': "Region VI (Western Visayas)",
    'REGION VII-CENTRAL VISAYAS': "Region VII (Central Visayas)",
    'REGION VII-EASTERN VISAYAS': "Region VIII (Eastern Visayas)",
    'REGION IX-ZAMBOANGA PENINSULA': "Region IX (Zamboanga Peninsula)",
    'REGION X-NORTHERN MINDANAO': "Region X (Northern Mindanao)",
    'REGION XI-DAVAO REGION': "Region XI (Davao Region)",
    'REGION XII-SOCCSKSARGEN': "Region XII (SOCCSKSARGEN)",
    'NATIONAL CAPITAL REGION': 'National Capital Region (NCR)',
    'CAR': 'Cordillera Administrative Region (CAR)',
    'CARAGA': 'Region XIII (Caraga)',
    'BARMM': 'Bangsamoro Autonomous Region In Muslim Mindanao (BARMM)'
})
df.head()

#### 'loc' column is inconsistent; contains cities AND provinces

In [ ]:
locs = df["loc"].unique()
locs

In [ ]:
# Dataset from provincial boundaries GeoJSON
provinces.head()

In [ ]:
province_names = provinces["adm2_en"].unique()
province_names

In [ ]:
provinces_up = [province.upper() for province in province_names]
check_dict = {}

for i in range(len(locs)):
    if locs[i] in provinces_up:
        check_dict[locs[i]] = True
    else:
        check_dict[locs[i]] = False

correct = []
wrong = []
for loc in check_dict.keys():
    if check_dict.get(loc):
        correct.append(loc)
    else:
        wrong.append(loc)
        
correct = np.array(correct)
wrong = np.array(wrong)

In [ ]:
# Values in loc that are provinces recorded in our other datasets
correct

In [ ]:
# Values in loc that are either cities or misspelled provinces
wrong

In [ ]:
df["loc"] = df["loc"].map({
    'DAGUPAN CITY': 'Pangasinan', 'NUEVA VISCAYA': 'Nueva Vizcaya', 'SANTIAGO CITY': 'Isabela', 'ANGELES CITY': 'Pampanga',
    'OLONGAPO CITY': 'Zambales', 'LUCENA CITY': 'Quezon', 'PUERTO PRINCESA CITY': 'Palawan', 'NAGA CITY': 'Camarines Sur',
    'BACOLOD CITY': 'Negros Occidental', 'ILOILO CITY': 'Iloilo', 'CEBU CITY': 'Cebu', 'LAPU-LAPU CITY': 'Cebu',
    'MANDAUE CITY': 'Cebu', ' NEGROS ORIENTAL': 'Negros Oriental', 'SISQUIJOR': 'Siquijor', 'ORMOC CITY': 'Leyte',
    'TACLOBAN': 'Leyte', 'ISABELA CITY': 'City of Isabela (Not a Province)', 'ZAMBOANGA CITY': 'Zamboanga del Sur',
    ' CAGAYAN DE ORO CITY': 'Misamis Oriental', 'ILIGAN CITY': 'Lanao del Norte', 'DAVAO CITY': 'Davao del Sur',
    'COTABATO CITY': 'Cotabato', 'GENERAL SANTOS CITY': 'South Cotabato', 'NORTH COTABATO': 'Cotabato',
    'MAGUINDANAO': 'Maguindanao del Norte', 'BAGUIO CITY': 'Benguet', 'BASILIG CITY': 'Surigao del Sur', 'BAYUGAN CITY': 'Agusan del Sur',
    'BUTUAN CITY': 'Agusan del Norte', 'CABADBARAN CITY': 'Agusan del Norte', 'SURIGAO CITY': 'Surigao del Norte',
    'TANDAG CITY': 'Surigao del Sur', 'CALOOCAN CITY': 'NCR, Third District (Not a Province)', 'LAS PINAS CITY': 'NCR, Fourth District (Not a Province)',
    'MALABON CITY': 'NCR, Third District (Not a Province)', 'MAKATI CITY': 'NCR, Fourth District (Not a Province)',
    'MANDALUYONG CITY': 'NCR, Second District (Not a Province)', 'MANILA CITY': 'NCR, City of Manila, First District (Not a Province)',
    'MARIKINA CITY': 'NCR, Second District (Not a Province)', 'MUNTINLUPA CITY': 'NCR, Fourth District (Not a Province)',
    'NAVOTAS CITY': 'NCR, Third District (Not a Province)', 'PARANAQUE CITY': 'NCR, Fourth District (Not a Province)',
    'PASAY CITY': 'NCR, Fourth District (Not a Province)', 'PASIG CITY': 'NCR, Second District (Not a Province)',
    'PATEROS': 'NCR, Fourth District (Not a Province)', 'QUEZON CITY': 'NCR, Second District (Not a Province)',
    'SAN JUAN CITY': 'NCR, Second District (Not a Province)', 'TAGUIG CITY': 'NCR, Fourth District (Not a Province)',
    'VALENZUELA CITY': 'NCR, Third District (Not a Province)', 'ILOCOS NORTE': 'Ilocos Norte', 'ILOCOS SUR': 'Ilocos Sur',
    'LA UNION': 'La Union', 'PANGASINAN': 'Pangasinan', 'BATANES': 'Batanes', 'CAGAYAN': 'Cagayan', 'ISABELA': 'Isabela',
    'QUIRINO': 'Quirino', 'AURORA': 'Aurora', 'BATAAN': 'Bataan', 'BULACAN': 'Bulacan', 'NUEVA ECIJA': 'Nueva Ecija',
    'PAMPANGA': 'Pampanga', 'TARLAC': 'Tarlac', 'ZAMBALES': 'Zambales', 'BATANGAS': 'Batangas', 'CAVITE': 'Cavite',
    'LAGUNA': 'Laguna', 'QUEZON': 'Quezon', 'RIZAL': 'Rizal', 'MARINDUQUE': 'Marinduque', 'OCCIDENTAL MINDORO': 'Occidental Mindoro',
    'ORIENTAL MINDORO': 'Oriental Mindoro', 'PALAWAN': 'Palawan', 'ROMBLON': 'Romblon', 'ALBAY': 'Albay',
    'CAMARINES NORTE': 'Camarines Norte', 'CAMARINES SUR': 'Camarines Sur', 'CATANDUANES': 'Catanduanes', 'MASBATE': 'Masbate',
    'SORSOGON': 'Sorsogon', 'AKLAN': 'Aklan', 'ANTIQUE': 'Antique', 'CAPIZ': 'Capiz', 'GUIMARAS': 'Guimaras', 'ILOILO': 'Iloilo',
    'NEGROS OCCIDENTAL': 'Negros Occidental', 'BOHOL': 'Bohol', 'CEBU': 'Cebu', 'BILIRAN': 'Biliran', 'EASTERN SAMAR': 'Eastern Samar',
    'LEYTE': 'Leyte', 'NORTHERN SAMAR': 'Northern Samar', 'SAMAR': 'Samar', 'SOUTHERN LEYTE': 'Southern Leyte', 'ZAMBOANGA DEL NORTE': 'Zamboanga del Norte',
    'ZAMBOANGA DEL SUR': 'Zamboanga del Sur', 'ZAMBOANGA SIBUGAY': 'Zamboanga Sibugay', 'BUKIDNON': 'Bukidnon',
    'CAMIGUIN': 'Camiguin', 'LANAO DEL NORTE': 'Lanao del Norte', 'MISAMIS OCCIDENTAL': 'Misamis Occidental', 'MISAMIS ORIENTAL': 'Misamis Oriental',
    'DAVAO DE ORO': 'Davao de Oro', 'DAVAO DEL NORTE': 'Davao del Norte', 'DAVAO DEL SUR': 'Davao del Sur', 'DAVAO OCCIDENTAL': 'Davao Occidental',
    'DAVAO ORIENTAL': 'Davao Oriental', 'SARANGANI': 'Sarangani', 'SOUTH COTABATO': 'South Cotabato', 'SULTAN KUDARAT': 'Sultan Kudarat',
    'BASILAN': 'Basilan', 'LANAO DEL SUR': 'Lanao del Sur', 'SULU': 'Sulu', 'TAWI-TAWI': 'Tawi-Tawi', 'ABRA': 'Abra', 'APAYAO': 'Apayao',
    'BENGUET': 'Benguet', 'IFUGAO': 'Ifugao', 'KALINGA': 'Kalinga', 'MOUNTAIN PROVINCE': 'Mountain Province', 'AGUSAN DEL NORTE': 'Agusan del Norte',
    'AGUSAN DEL SUR': 'Agusan del Sur', 'DINAGAT ISLANDS': 'Dinagat Islands', 'SURIGAO DEL NORTE': 'Surigao del Norte', 'SURIGAO DEL SUR': 'Surigao del Sur'})

df.rename(columns = {'loc':'province'}, inplace = True)
df.head()

In [ ]:
# Checking if all entries are ACTUALLY provinces
complete = True
for province in df["province"].unique():
    if province in province_names:
        pass
    else:
        print(province)
        complete = False
        break

print(f"Complete: {complete}")

#### Unique codes for provinces and regions

In [ ]:
region_codes = {}

for i in range(regions.shape[0]):
    region_codes[regions.iloc[i]['adm1_en']] = regions.iloc[i]['adm1_psgc']

# region_codes

In [ ]:
province_codes = {}

for i in range(provinces.shape[0]):
    province_codes[provinces.iloc[i]['adm2_en']] = provinces.iloc[i]['adm2_psgc']

# province_codes

In [ ]:
province_names = df['province'].to_list()
region_names = df['region'].to_list()
print(province_names[0:5])
print(region_names[0:5])

In [ ]:
p_codes = []
r_codes = []

for i in range(len(province_names)):
    p_codes.append(province_codes.get(province_names[i]))
    r_codes.append(region_codes.get(region_names[i]))
    
print(p_codes[0:5])
print(r_codes[0:5])

In [ ]:
df.head()

In [ ]:
df.insert(1, "adm2_psgc", p_codes)
df.insert(6, "adm1_psgc", r_codes)
df.head()

#### Getting month names and order

In [ ]:
dates = pd.to_datetime(df['date'], dayfirst=True)
months_name = []
months_num = []

month_dict = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

for date in dates:
    months_num.append(date.month)
    months_name.append(month_dict.get(date.month))

df.insert(5, "month_num", months_num)
df.insert(5, "month_name", months_name)

In [ ]:
df.head()

#### Saving the cleaned dataset

In [ ]:
df = df[["adm1_psgc", "adm2_psgc", "region", "province", "date", "month_name", "month_num",
   "cases", "deaths"]]

In [ ]:
path = "../datasets/csv/dengue_cleaned.csv"
df.to_csv(path)

In [ ]:
# Checking
df = pd.read_csv(path, index_col=0)
df.head()

---
### Exploratory Data Analysis
<a href="#Table-of-Contents">Back to Table of Contents</a>

#### List of Graphs
<ul>
    <li><a href="#Cases-and-Deaths-per-region">Cases and Deaths per region</a></li>
    <li><a href="#Cases-and-Deaths-per-province">Cases and Deaths per province</a></li>
    <li><a href="#Fatality-Rate-per-region">Fatality Rate per region</a></li>
    <li><a href="#Fatality-Rate-per-province">Fatality Rate per province</a></li>
    <li><a href="#Cases-per-month">Cases per month</a></li>
    <li><a href="#Deaths-per-month">Deaths per month</a></li>
    <li><a href="#Cases-and-Deaths-over-time-(regional)">Cases and Deaths over time (regional)</a></li>
    <li><a href="#Cases-and-Deaths-over-time-(national)">Cases and Deaths over time (national)</a></li>
    <li><a href="#Fatality-rate-Choropleth-Map">Fatality rate Choropleth Map</a></li>
</ul>

##### Preparation: Mapping long names to shorter versions

In [ ]:
df['region'] = df['region'].map({
    'Bangsamoro Autonomous Region In Muslim Mindanao (BARMM)': 'BARMM',
    'Cordillera Administrative Region (CAR)': 'CAR',
    'MIMAROPA Region': 'MIMAROPA',
    'National Capital Region (NCR)': 'NCR',
    'Region I (Ilocos Region)': 'Region I',
    'Region II (Cagayan Valley)': 'Region II',
    'Region III (Central Luzon)': 'Region III',
    'Region IV-A (CALABARZON)': 'Region IV-A',
    'Region IX (Zamboanga Peninsula)': 'Region IX',
    'Region V (Bicol Region)': 'Region V',
    'Region VI (Western Visayas)': 'Region VI',
    'Region VII (Central Visayas)': 'Region VII',
    'Region VIII (Eastern Visayas)': 'Region VIII',
    'Region X (Northern Mindanao)': 'Region X',
    'Region XI (Davao Region)': 'Region XI',
    'Region XII (SOCCSKSARGEN)': 'Region XII',
    'Region XIII (Caraga)': 'Region XIII'
})

In [ ]:
color_dict = {
    'BARMM': '#5B7FA4',
    'CAR': '#A27F5D',
    'MIMAROPA': '#00FF85',
    'NCR': '#585858',
    'Region I': '#2000FF',
    'Region II': '#DFFF00',
    'Region III': '#E100FF',
    'Region IV-A': '#1EFF00',
    'Region IX': '#FF0026',
    'Region V': '#00FFD9',
    'Region VI': '#FF9100',
    'Region VII': '#9A6595',
    'Region VIII': '#FF4CD5',
    'Region X': '#48B7AC',
    'Region XI': '#9D609F',
    'Region XII': '#629F60',
    'Region XIII': '#9FC13E'
}

#### Cases and Deaths per region
<a href="#List-of-Graphs">Back to list</a>

In [ ]:
cases_deaths_per_region = df.groupby(["region", "adm1_psgc"])[["cases", "deaths"]].sum().reset_index()
cases_deaths_per_region.head()

In [ ]:
fig = px.bar(cases_deaths_per_region.sort_values(ascending=False, by="cases"),
             x="region",
             y="cases",
             color="cases",
             color_continuous_scale=px.colors.sequential.OrRd,
             labels = {
                "region": "Region",
                "cases": "Number of Cases"
             },
             text_auto=True
            )
fig.show()

In [ ]:
fig = px.bar(cases_deaths_per_region.sort_values(ascending=False, by="deaths"),
             x="region",
             y="deaths",
             color="deaths",
             color_continuous_scale=px.colors.sequential.OrRd,
             labels = {
                "region": "Region",
                "deaths": "Number of Deaths"
             },
             text_auto=True
            )
fig.show()

#### Cases and Deaths per province
<a href="#List-of-Graphs">Back to list</a>

In [ ]:
cases_deaths_per_province = df.groupby(["province", "adm2_psgc",  "region", "adm1_psgc"])[["cases", "deaths"]].sum().reset_index()
cases_deaths_per_province.head()

In [ ]:
'''
Maguindanao del Sur (code 1908800000) and Special Geographic Area (1909900000) were NOT
included in the Dengue dataset, because official formation of these "provinces" happened
AFTER the Dengue dataset was recorded (at 2020).

To remedy any future indexing errors, Maguindanao del Sur will copy values from Maguinadanao del Norte,
and the Special Geographic Area will copy from Cotabato, mirroring the cases and deaths of their
geographical neighbors.
'''
temp_dict = {
    "province": ["Maguindanao del Sur", "Special Geographic Area"],
    "adm2_psgc": [1908800000, 1909900000],
    "region": ["BARMM", "BARMM"],
    "adm1_psgc": [1900000000, 1900000000],
    "cases": [264, 1485],
    "deaths": [7, 88]
}

temp_df = pd.DataFrame(temp_dict, index=[0, 1])
cases_deaths_per_province = pd.concat([cases_deaths_per_province, temp_df], ignore_index=True)
cases_deaths_per_province.tail()

In [ ]:
# Filtering option
specific_regions = ["NCR", "Region XII"]
condition = f"(cases_deaths_per_province['region'] == '{specific_regions[0]}')"

for region in specific_regions[1:]:
    condition += f" | (cases_deaths_per_province['region'] == '{region}')"

condition

In [ ]:
fig = px.bar(cases_deaths_per_province.sort_values(ascending=False, by="cases").loc[eval(condition)],
             x="province",
             y="cases",
             color="region",
             color_discrete_map=color_dict,
             labels = {
                "province": "Province",
                "cases": "Number of Cases",
                "region": "Region"
             },
             text_auto=True
            )
fig.show()

In [ ]:
fig = px.bar(cases_deaths_per_province.sort_values(ascending=False, by="deaths").loc[eval(condition)],
             x="province",
             y="deaths",
             color="region",
             color_discrete_map=color_dict,
             labels = {
                "province": "Province",
                "deaths": "Number of Deaths",
                "region": "Region"
             },
             text_auto=True
            )
fig.show()

#### Fatality Rate per region
<a href="#List-of-Graphs">Back to list</a>

In [ ]:
cases_deaths_per_region["fatality_rate"] = round((cases_deaths_per_region["deaths"] / cases_deaths_per_region["cases"]) * 100.00, 2)
cases_deaths_per_region.head()
cases_deaths_per_region.to_csv("../datasets/csv/cases_deaths_per_region.csv")

In [ ]:
fig = px.bar(cases_deaths_per_region.sort_values(ascending=False, by="fatality_rate"),
             x="region",
             y="fatality_rate",
             color="region",
             color_discrete_map=color_dict,
             labels = {
                "region": "Region",
                "fatality_rate": "Fatality Rate (in %)"
             },
             text_auto=True
            )
fig.show()

#### Fatality Rate per province
<a href="#List-of-Graphs">Back to list</a>

In [ ]:
cases_deaths_per_province["fatality_rate"] = round((cases_deaths_per_province["deaths"] / cases_deaths_per_province["cases"]) * 100.00, 2)
cases_deaths_per_province.head()
cases_deaths_per_province.to_csv("../datasets/csv/cases_deaths_per_province.csv")

In [ ]:
# Filtering option
specific_regions = ["CAR", "NCR", "Region XII"]
condition = f"(cases_deaths_per_province['region'] == '{specific_regions[0]}')"

for region in specific_regions[1:]:
    condition += f" | (cases_deaths_per_province['region'] == '{region}')"

condition

In [ ]:
fig = px.bar(cases_deaths_per_province.sort_values(ascending=False, by="fatality_rate").loc[eval(condition)],
             x="province",
             y="fatality_rate",
             color="region",
             color_discrete_map=color_dict,
             labels = {
                "province": "Province",
                "fatality_rate": "Fatality Rate (in %)",
                "region": "Region"
             },
             text_auto=True
            )
fig.show()

#### Cases per month
<a href="#List-of-Graphs">Back to list</a>

In [ ]:
per_month = df.groupby(["month_num", "month_name"])[["cases", "deaths"]].sum().reset_index()
per_month.head()

In [ ]:
fig = px.bar(per_month.sort_values(by="month_num"),
             x="month_name",
             y="cases",
             labels = {
                "month_name": "Month",
                "cases": "Number of Cases (in thousands)"
             },
             text_auto=True
            )
fig.show()

#### Deaths per month
<a href="#List-of-Graphs">Back to list</a>

In [ ]:
fig = px.bar(per_month.sort_values(by="month_num"),
             x="month_name",
             y="deaths",
             labels = {
                "month_name": "Month",
                "deaths": "Number of Deaths"
             },
             text_auto=True
            )
fig.show()

#### Cases and Deaths over time (regional)
<a href="#List-of-Graphs">Back to list</a>

In [ ]:
df.head()

In [ ]:
per_month_per_region = df[['region', 'month_num', 'month_name', 'cases', 'deaths']].groupby(['region', 'month_num', 'month_name'])[['cases', 'deaths']].sum().reset_index()
per_month_per_region

In [ ]:
fig = px.line(per_month_per_region,
              x="month_name",
              y="cases",
              color="region",
              labels = {
                  'month_name': 'Month',
                  'cases': 'Number of Cases',
                  'region': 'Region'
              }
             )
fig.show()

In [ ]:
fig = px.line(per_month_per_region,
              x="month_name",
              y="deaths",
              color="region",
              labels = {
                  'month_name': 'Month',
                  'deaths': 'Number of Deaths',
                  'region': 'Region'
              }
             )
fig.show()

#### Cases and Deaths over time (national)
<a href="#List-of-Graphs">Back to list</a>

In [ ]:
per_month = per_month_per_region[['month_num', 'month_name', 'cases', 'deaths']].groupby(['month_num', 'month_name'])[['cases', 'deaths']].sum().reset_index()
per_month

In [ ]:
fig = px.line(per_month,
              x="month_name",
              y="cases",
              labels = {
                  'month_name': 'Month',
                  'cases': 'Number of Cases (in thousands)'
              }
             )
fig.show()

#### Fatality rate Choropleth Map
<a href="#List-of-Graphs">Back to list</a>

In [ ]:
import mapboxgl
mapboxgl.access_token = 'pk.eyJ1IjoicjNuejNsIiwiYSI6ImNsdTNlZ2w2ODB6MjIyanFodDR6NXl2MnMifQ.kveM31v-0p75V41iiEt3bQ'

In [ ]:
gdf = provinces[["adm2_psgc", "geometry"]].merge(cases_deaths_per_province, on="adm2_psgc").set_index("adm2_psgc")

fig = px.choropleth_mapbox(
    gdf,
    geojson=gdf.geometry,
    locations=gdf.index,
    color="fatality_rate",
    color_continuous_scale=px.colors.sequential.OrRd,
    center={"lat": 14.6061, "lon": 120.9803},
    mapbox_style="open-street-map",
    zoom=8.0
)

fig.show()

In [ ]:
regions.head(1)

In [ ]:
gdf = regions[["adm1_psgc", "geometry"]].merge(cases_deaths_per_region, on="adm1_psgc").set_index("adm1_psgc")

fig = px.choropleth_mapbox(
    gdf,
    geojson=gdf.geometry,
    locations=gdf.index,
    color="fatality_rate",
    color_continuous_scale=px.colors.sequential.OrRd,
    center={"lat": 14.6061, "lon": 120.9803},
    mapbox_style="open-street-map",
    zoom=8.0
)

fig.show()

#### Test App

In [ ]:
external_stylesheets = [dbc.themes.MINTY]
app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
    
    dbc.Row([
        dcc.Graph(
            figure = fig.update_layout(
                margin=dict(b=0),
                height=350
            )
        )
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                figure = px.bar(per_month.sort_values(by="month_num"),
                                x="month_name",
                                y="deaths",
                                labels = {
                                    "month_name": "Month",
                                    "deaths": "Number of Deaths"
                                },
                                text_auto=True).update_layout(
                    margin=dict(t=0, b=50),
                    height=350
                )
            )
        ], width=5),
        dbc.Col([
        ], width=7)
    ])
    
], fluid=True)

# Comment out if you want to run the app in the notebook
# Further testing will be done using app.py (see main directory or repository)
'''
if __name__ == "__main__":
    app.run()
# '''

In [ ]:
regions

In [ ]:
hospitals = gpd.read_file("../datasets/shapefiles/hospitals/hospitals.shp")
hospitals.head()

In [ ]:
temp_reg_code = {'REGION IX (ZAMBOANGA PENINSULA)': 900000000,
 'Bangsamoro Autonomous Region In Muslim Mindanao (BARMM)': 1900000000,
 'Region X (northern Mindanao)': 1000000000,
 'REGION XII (SOCCSKSARGEN)': 1200000000,
 'REGION VII (CENTRAL VISAYAS)': 700000000,
 'CAR (CORDILLERA ADMINISTRATIVE REGION': 1400000000,
 'REGION VI (WESTERN VISAYAS)': 600000000,
 'REGION IV-B (MIMAROPA)': 1700000000,
 'REGION VIII (EASTERN VISAYAS)': 800000000,
 'REGION II (CAGAYAN VALLEY)': 200000000,
 'REGION XIII (CARAGA)': 1600000000,
 'REGION III (CENTRAL LUZON)': 300000000,
 'REGION V (BICOL REGION)': 500000000,
 'REGION I (ILOCOS REGION)': 100000000,
 'REGION IV-A (CALABARZON)': 400000000,
 'NCR (NATIONAL CAPITAL REGION)': 1300000000,
 'REGION XI (DAVAO REGION)': 1100000000}

temp_reg_name = {'REGION IX (ZAMBOANGA PENINSULA)': 'Region IX (Zamboanga Peninsula)',
 'Region X (northern Mindanao)': 'Region X (Northern Mindanao)',
 'REGION XII (SOCCSKSARGEN)': 'Region XII (SOCCSKSARGEN)',
 'REGION VII (CENTRAL VISAYAS)': 'Region VII (Central Visayas)',
 'CAR (CORDILLERA ADMINISTRATIVE REGION': 'Cordillera Administrative Region (CAR)',
 'REGION VI (WESTERN VISAYAS)': 'Region VI (Western Visayas)',
 'REGION IV-B (MIMAROPA)': 'MIMAROPA Region',
 'REGION VIII (EASTERN VISAYAS)': 'Region VIII (Eastern Visayas)',
 'REGION II (CAGAYAN VALLEY)': 'Region II (Cagayan Valley)',
 'REGION XIII (CARAGA)': 'Region XIII (Caraga)',
 'REGION III (CENTRAL LUZON)': 'Region III (Central Luzon)',
 'REGION V (BICOL REGION)': 'Region V (Bicol Region)',
 'REGION I (ILOCOS REGION)': 'Region I (Ilocos Region)',
 'REGION IV-A (CALABARZON)': 'Region IV-A (CALABARZON)',
 'NCR (NATIONAL CAPITAL REGION)': 'National Capital Region (NCR)',
 'REGION XI (DAVAO REGION)': 'Region XI (Davao Region)'}

temp_list = []
for i in range(hospitals.shape[0]):
    temp_list.append(temp_reg_code.get(hospitals.iloc[i]["region"]))

hospitals["region"] = hospitals["region"].map(temp_reg_name)
hospitals.insert(7, "adm1_psgc", temp_list)

In [ ]:
hospitals.head()

In [ ]:
hospitals_gdf = hospitals[["healthfaci", "adm1_psgc", "geometry"]].merge(
    cases_deaths_per_province, on="adm1_psgc").set_index("adm1_psgc")

In [ ]:
fig = px.scatter_mapbox(hospitals_gdf, lat=hospitals_gdf.geometry.y, lon=hospitals_gdf.geometry.x,  center={"lat": 12.74, "lon": 120.9803}, mapbox_style="open-street-map", zoom=4.335,
                       hover_name="healthfaci", hover_data=["cases", "deaths", "fatality_rate"],
                       labels={
                           "cases": "Number of Cases",
                           "deaths": "Number of Deaths",
                           "fatality_rate": "Fatality_rate (in %)"
                       })
fig.show()